# T4 Combine Detection Results

- import the t1 inferenced point clouds
- import the t3 doors 
- export graphs

## LIBRARIES

In [1]:

#IMPORT PACKAGES
from rdflib import Graph, URIRef
import os.path
import importlib
from pathlib import Path
import numpy as np
import xml.etree.ElementTree as ET
import open3d as o3d
import uuid    
import pye57 
import ifcopenshell
import ifcopenshell.geom as geom
import ifcopenshell.util
from ifcopenshell.util.selector import Selector
import multiprocessing
import random as rd
import pandas as pd
# from tabulate import tabulate
import cv2
import laspy

import geomapi
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl
import geomapi.tools.progresstools as pt

#import utils
import context 
import utils as utl
import utils.t1_utils as t1
import utils.t2_utils as t2
import utils.t4_utils as t4

import json

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

## INPUTS

In [4]:
#paths
path=Path(os.getcwd()).parents[2]

print(path)
input_folder=path/'data'/'t2'/'test' 
# input_folder=path/'data'/'t2'/'training' 

class_file=path/'data'/'_classes.json'
output_folder=path/'data'/'t4'/ 'test'
# output_folder=path/'data'/'t4'/ 'training'

os.makedirs(output_folder, exist_ok=True)

#parameters
resolution=0.05
min_cluster_points=1000
eps=0.5

#doors
threshold_door_dim=1.6#m

#walls
threshold_clustering_distance=0.4
threshold_min_cluster_points=500
threshold_wall_verticality=0.2# angle to horizontal
threshold_wall_dim=0.5
size=[12,12,100] #size wall boxes

#columns
threshold_column_points=1000
threshold_column_height=1.5#m
threshold_column_verticality=0.2# angle to horizontal


c:\Users\u0094523\OneDrive - KU Leuven\2024-05 CVPR scan-to-BIM challenge


### Import classes

In [5]:
# Read the JSON file
with open(class_file, 'r') as file:
    json_data = json.load(file)

# Create a dictionary
class_dict = {
    'classes': json_data['classes'],
    'default': json_data['default'],
    'type': json_data['type'],
    'format': json_data['format'],
    'created_with': json_data['created_with']
}
print(class_dict)

{'classes': [{'name': 'unassigned', 'id': 255, 'temp_id': 0, 'color': '#9da2ab'}, {'name': 'floors', 'id': 0, 'temp_id': 1, 'color': '#03c2fc'}, {'name': 'ceilings', 'id': 1, 'temp_id': 2, 'color': '#e81416'}, {'name': 'walls', 'id': 2, 'temp_id': 3, 'color': '#ffa500'}, {'name': 'columns', 'id': 3, 'temp_id': 4, 'color': '#faeb36'}, {'name': 'doors', 'id': 4, 'temp_id': 5, 'color': '#79c314'}, {'name': 'beams', 'id': 5, 'temp_id': 6, 'color': '#79c314'}], 'default': 255, 'type': 'semantic_segmentation', 'format': 'kitti', 'created_with': {'name': 'Saiga', 'version': '1.0.1'}}


## CREATE INITIAL OBJECT PCDS

In [6]:
files=utl.get_list_of_files(input_folder, '.laz')

In [17]:
# for f in files[:1]:
for f in files:
    
    objectNodes=[]
    
    # check if las/pcd variable is already defined    
    print(f'processing {ut.get_filename(f)}...')      
    las = laspy.read(f) #if 'las' not in globals() else las
    pcd=gmu.las_to_pcd(las,getNormals=True) #if 'pcd' not in globals() else pcd
    
    #create thrash node
    c=[c for c in class_dict['classes'] if c['id']==255][0]    
    thrashNode=t4.create_thrash_node(las,pcd,f,c)
    
    #CLUSTER OBJECTS
    for c in class_dict['classes']:
        print(c['name'])
        
        ##------------------------FLOORS CEILINGS--------------------------------------------
        if c['id'] in [0,1]: #floors,ceilings CORRECT
            nodes=t4.create_floor_and_ceiling_nodes(las,pcd,f,c,sample_resolution=resolution,
                                                                                    distance_threshold=0.05, 
                                                                                    min_inliers=1000,
                                                                                    eps=eps,
                                                                                    min_cluster_points=200) #this somehow created only 2 nodes in 05_MedOffice_01_F2 while there should be more nodes
            objectNodes.extend(nodes)

            print(f': {len(nodes)} clusters found')  
        
        ##------------------------WALLS--------------------------------------------
        if c['id'] in [2]: 
            nodes,rest_pcd=t4.create_wall_nodes(las,pcd,f,c,sample_resolution=0.03,
                                                distance_threshold=0.03, 
                                                min_inliers=200,
                                                eps=eps,
                                                threshold_min_cluster_points=threshold_min_cluster_points,
                                                size=size,
                                                threshold_wall_verticality=threshold_wall_verticality,
                                                threshold_wall_dim=threshold_wall_dim,
                                                threshold_clustering_distance=threshold_clustering_distance)
            nodes=t4.merge_wall_nodes2(nodes,
                                       threshold_clustering_distance=threshold_clustering_distance,
                                        threshold_clustering_orthogonal_distance= 0.2,
                                        threshold_clustering_coplanar_distance = 5)
            
            objectNodes.extend(nodes)
            thrashNode.resource+=rest_pcd
            print(f': {len(nodes)} clusters found')
     
            
        # ###------------------------COLUMNS--------------------------------------------
        if c['id'] in [3]:
            nodes,rest_pcd=t4.create_column_nodes(las,pcd,f,c,eps=1,min_cluster_points=250,
                                threshold_column_verticality=threshold_column_verticality,
                                threshold_column_height=threshold_column_height,
                                threshold_column_points=threshold_column_points)
            objectNodes.extend(nodes)
            thrashNode.resource+=rest_pcd
    
            print( f': {len(nodes)} clusters found')       
            
        # ###------------------------DOORS--------------------------------------------
        if c['id'] in [4]: 
            nodes,rest_pcd=t4.create_door_nodes(las,pcd,f,c,
                                            eps=0.5,
                                            min_cluster_points=200,
                                            threshold_door_dim=0.5)
            objectNodes.extend(nodes)
            thrashNode.resource+=rest_pcd
       
            print(f': {len(nodes)} clusters found') 
    
    print(f'Created {len(objectNodes)} PointCloudNodes created from {ut.get_filename(f)}')
    
    
    #EXPORT RESULTS
    #merge objects
    total_pcd_nodes=objectNodes +[thrashNode]
    joined_pcd=gmu.join_geometries([n.resource for n  in total_pcd_nodes])

    #export graph
    tl.nodes_to_graph(total_pcd_nodes,
                    graphPath=str(output_folder/f'{ut.get_filename(f)}.ttl'),
                    save=True)
    
    #obtain labels
    labels_segmentation=[]
    labels_objects=[]
    for i,n in enumerate(total_pcd_nodes):
        length=len(np.asarray(n.resource.points))
        labels_segmentation.extend(list(np.full(length,n.class_id)))
        labels_objects.extend(list(np.full(length,n.object_id)))  
    labels_classes=np.array(labels_segmentation)
    labels_objects=np.array(labels_objects)
    
    #create a new las file with the labels
    hdr = laspy.LasHeader(version="1.4", point_format=2)
    las2 = laspy.LasData(hdr)
    las2.x=np.asarray(joined_pcd.points)[:,0]
    las2.y=np.asarray(joined_pcd.points)[:,1]
    las2.z=np.asarray(joined_pcd.points)[:,2]
    las2.red= (np.asarray(joined_pcd.colors)[:,0] * 65535).astype(np.uint16)
    las2.green= (np.asarray(joined_pcd.colors)[:,1] * 65535).astype(np.uint16)
    las2.blue= (np.asarray(joined_pcd.colors)[:,2] * 65535).astype(np.uint16)

    gmu.las_add_extra_dimensions(las2,(labels_classes,labels_objects),['classes','objects'],['uint8','uint16'])
    # Write the LAS file as LAZ
    las2.write(str(output_folder/f'{ut.get_filename(f)}.laz'))
    print(str(output_folder/f'{ut.get_filename(f)}.laz'))
    print('DONE')


processing 08_ShortOffice_01_F1_small_pred...
unassigned
floors
Function fit_planes took 2.8139 seconds to execute.
: 4 clusters found
ceilings
Function fit_planes took 4.3367 seconds to execute.
: 12 clusters found
walls
Function fit_planes took 3.7314 seconds to execute.
Function split_point_cloud_in_planar_clusters2 took 4.5993 seconds to execute.
Function fit_planes took 3.8452 seconds to execute.
Function split_point_cloud_in_planar_clusters2 took 4.5934 seconds to execute.
Function fit_planes took 4.3375 seconds to execute.
Function split_point_cloud_in_planar_clusters2 took 5.1534 seconds to execute.
Function fit_planes took 6.6976 seconds to execute.
Function split_point_cloud_in_planar_clusters2 took 8.2009 seconds to execute.
Function fit_planes took 6.0423 seconds to execute.
Function split_point_cloud_in_planar_clusters2 took 7.5966 seconds to execute.
Function fit_planes took 2.7733 seconds to execute.
Function split_point_cloud_in_planar_clusters2 took 3.3397 seconds to e

In [15]:
joined_pcd=gmu.join_geometries([p.resource.paint_uniform_color(ut.random_color()) for p in objectNodes if p.class_id==2 ])
o3d.visualization.draw_geometries([joined_pcd])

In [90]:
import copy
stored_nodes=copy.deepcopy(nodes)

In [ ]:
nodes=copy.deepcopy(stored_nodes)

## VISUALIZE

In [78]:

joined_pcd=gmu.join_geometries([p.paint_uniform_color(ut.random_color()) for p in clustered_pcds ])
joined_meshes=gmu.join_geometries([p.paint_uniform_color(ut.random_color()) for p in clustered_plane_meshes ])
o3d.visualization.draw_geometries([joined_pcd,joined_meshes])

In [ ]:
# {key:value for key, value in objectNodes[1].__dict__.items() if not key.startswith('__') and not callable(key)}              

In [ ]:
# pcd_slice=t2.slice_point_cloud(pcd, -100, pcd.get_center()[2])
# joined_pcd=gmu.join_geometries([p.resource.paint_uniform_color(ut.random_color()) for p in objectNodes])
# o3d.visualization.draw_geometries([joined_pcd,gmu.sample_geometry(pcd_slice)[0]])

## CREATE GRAPHS TRAINING DATA

In [5]:
#paths
path=Path(os.getcwd()).parents[2]

print(path)
input_folder=path/'data'/'t1'/'train' 
class_file=path/'data'/'_classes.json'
output_folder=path/'data'/'t4'/ 'train'
os.makedirs(output_folder, exist_ok=True)

c:\Users\u0094523\OneDrive - KU Leuven\2024-05 CVPR scan-to-BIM challenge


In [15]:
files=utl.get_list_of_files(input_folder,'.laz')

for f in files: 
    pcdNodes=[]
    #import pcd and check if las/pcd variable is already defined    
    print(f'processing {ut.get_filename(f)} ...')      
    las = laspy.read(f) #○if 'las' not in globals() else las
    pcd=gmu.las_to_pcd(las) #if 'pcd' not in globals() else pcd # this is the slowest step
        
    #match pcd to nodes
    for c in class_dict['classes']:
        idx=np.where((las['classes']==c['id']))[0]
        class_pcd=pcd.select_by_index(idx)
        object_labels=las['objects'][idx]
        counter=0
        for j in np.unique(object_labels):
            indices=np.where(object_labels==j)[0]
            object_pcd=class_pcd.select_by_index(indices)
            pcdNodes.append(PointCloudNode(resource=object_pcd,
                                                class_id=c['id'],
                                                class_name=c['name'],
                                                object_id=c['id']*1000+counter, 
                                                color=ut.random_color(),
                                                name=ut.get_filename(f)+'_'+c['name']+'_'+str(c['id']*1000+counter))) 
            counter+=1
    print(f'{len(pcdNodes)} Nodes found')     
        

   #export graph
    tl.nodes_to_graph(pcdNodes,
                    graphPath=str(output_folder/f'{ut.get_filename(f)}.ttl'),
                    save=True)
                                                
    
    #obtain labels
    joined_pcd=gmu.join_geometries([n.resource for n  in pcdNodes])
    labels_segmentation=[]
    labels_objects=[]
    for i,n in enumerate(pcdNodes):
        length=len(np.asarray(n.resource.points))
        labels_segmentation.extend(list(np.full(length,n.class_id)))
        labels_objects.extend(list(np.full(length,n.object_id)))  
    labels_classes=np.array(labels_segmentation)
    labels_objects=np.array(labels_objects)
    
    #create a new las file with the labels
    hdr = laspy.LasHeader(version="1.4", point_format=2)
    las2 = laspy.LasData(hdr)
    las2.x=np.asarray(joined_pcd.points)[:,0]
    las2.y=np.asarray(joined_pcd.points)[:,1]
    las2.z=np.asarray(joined_pcd.points)[:,2]
    las2.red= (np.asarray(joined_pcd.colors)[:,0] * 65535).astype(np.uint16)
    las2.green= (np.asarray(joined_pcd.colors)[:,1] * 65535).astype(np.uint16)
    las2.blue= (np.asarray(joined_pcd.colors)[:,2] * 65535).astype(np.uint16)

    gmu.las_add_extra_dimensions(las2,(labels_classes,labels_objects),['classes','objects'],['uint8','uint32'])
    # Write the LAS file as LAZ
    las2.write(str(output_folder/f'{ut.get_filename(f)}.laz'))
    print(str(output_folder/f'{ut.get_filename(f)}.laz'))
    print('DONE')

processing 05_MedOffice_01_F2_small1 ...
289 Nodes found
c:\Users\u0094523\OneDrive - KU Leuven\2024-05 CVPR scan-to-BIM challenge\data\t4\train\05_MedOffice_01_F2_small1.laz
DONE
processing 19_MedOffice_07_F4_small1 ...
206 Nodes found
c:\Users\u0094523\OneDrive - KU Leuven\2024-05 CVPR scan-to-BIM challenge\data\t4\train\19_MedOffice_07_F4_small1.laz
DONE
processing 32_ShortOffice_05_F1_small1 ...
174 Nodes found
c:\Users\u0094523\OneDrive - KU Leuven\2024-05 CVPR scan-to-BIM challenge\data\t4\train\32_ShortOffice_05_F1_small1.laz
DONE
processing 32_ShortOffice_05_F2_small1 ...
219 Nodes found
c:\Users\u0094523\OneDrive - KU Leuven\2024-05 CVPR scan-to-BIM challenge\data\t4\train\32_ShortOffice_05_F2_small1.laz
DONE
processing 32_ShortOffice_05_F3_small1 ...
214 Nodes found
c:\Users\u0094523\OneDrive - KU Leuven\2024-05 CVPR scan-to-BIM challenge\data\t4\train\32_ShortOffice_05_F3_small1.laz
DONE
processing 33_SmallBuilding_03_F1_small1 ...
277 Nodes found
c:\Users\u0094523\OneDrive 